# Data Preparation



In [1]:
# Load libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error



In [2]:
# Load data
sales_train = pd.read_csv('sales_train.csv')
calendar = pd.read_csv('calendar.csv')
items = pd.read_csv('items_weekly_sell_prices.csv')
events = pd.read_csv('calendar_events.csv')

In [3]:
items = items.iloc[:len(sales_train)]

In [4]:

row_count = items.shape[0]
print("Number of rows:", row_count)

Number of rows: 30490


In [5]:
train_data_sales = pd.concat([sales_train, items], axis=1)



In [6]:
print(train_data_sales.columns)

Index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'd_1',
       'd_2', 'd_3', 'd_4',
       ...
       'd_1536', 'd_1537', 'd_1538', 'd_1539', 'd_1540', 'd_1541', 'store_id',
       'item_id', 'wm_yr_wk', 'sell_price'],
      dtype='object', length=1551)


In [7]:
print("Head:")
print(train_data_sales.head())

# View the last few rows using the tail() function
print("\nTail:")
print(items.tail())

Head:
                              id        item_id    dept_id   cat_id  store_id  \
0  HOBBIES_1_001_CA_1_evaluation  HOBBIES_1_001  HOBBIES_1  HOBBIES      CA_1   
1  HOBBIES_1_002_CA_1_evaluation  HOBBIES_1_002  HOBBIES_1  HOBBIES      CA_1   
2  HOBBIES_1_003_CA_1_evaluation  HOBBIES_1_003  HOBBIES_1  HOBBIES      CA_1   
3  HOBBIES_1_004_CA_1_evaluation  HOBBIES_1_004  HOBBIES_1  HOBBIES      CA_1   
4  HOBBIES_1_005_CA_1_evaluation  HOBBIES_1_005  HOBBIES_1  HOBBIES      CA_1   

  state_id  d_1  d_2  d_3  d_4  ...  d_1536  d_1537  d_1538  d_1539  d_1540  \
0       CA    0    0    0    0  ...       1       0       1       0       0   
1       CA    0    0    0    0  ...       0       0       0       0       1   
2       CA    0    0    0    0  ...       0       0       0       0       0   
3       CA    0    0    0    0  ...       2       3       1       1       3   
4       CA    0    0    0    0  ...       2       1       1       2       2   

   d_1541  store_id        item_

In [8]:
import pandas as pd

# Merge training data with calendar data
merged_data = train_data_sales.merge(calendar, how='outer')


In [9]:
print("Head:")
print(merged_data.head())


Head:
                              id        item_id    dept_id   cat_id  store_id  \
0  HOBBIES_1_001_CA_1_evaluation  HOBBIES_1_001  HOBBIES_1  HOBBIES      CA_1   
1  HOBBIES_1_001_CA_1_evaluation  HOBBIES_1_001  HOBBIES_1  HOBBIES      CA_1   
2  HOBBIES_1_001_CA_1_evaluation  HOBBIES_1_001  HOBBIES_1  HOBBIES      CA_1   
3  HOBBIES_1_001_CA_1_evaluation  HOBBIES_1_001  HOBBIES_1  HOBBIES      CA_1   
4  HOBBIES_1_001_CA_1_evaluation  HOBBIES_1_001  HOBBIES_1  HOBBIES      CA_1   

  state_id  d_1  d_2  d_3  d_4  ...  d_1538  d_1539  d_1540  d_1541  store_id  \
0       CA    0    0    0    0  ...       1       0       0       1      CA_1   
1       CA    0    0    0    0  ...       1       0       0       1      CA_1   
2       CA    0    0    0    0  ...       1       0       0       1      CA_1   
3       CA    0    0    0    0  ...       1       0       0       1      CA_1   
4       CA    0    0    0    0  ...       1       0       0       1      CA_1   

         item_id  wm

In [10]:
# Select relevant columns
selected_columns = ['item_id', 'store_id', 'date', 'sell_price'] + [f'd_{i}' for i in range(1, 250)]
sales_data = merged_data[selected_columns]

In [11]:
# Check for duplicate column names
duplicate_columns = sales_data.columns[sales_data.columns.duplicated()]
if len(duplicate_columns) > 0:
    sales_data.columns = sales_data.columns.where(~sales_data.columns.duplicated(), 
                                                  sales_data.columns + '_sales')



In [12]:
# Reshape data to have one row per sale
sales_data_train = sales_data.melt(id_vars=['item_id', 'store_id', 'date', 'sell_price'],
                              value_vars=sales_data.columns[4:],
                              var_name='day', value_name='sales')

In [16]:
# Convert 'date' column to datetime format
sales_data.loc[:, 'date'] = pd.to_datetime(sales_data['date'])


C:\Users\Admin\AppData\Local\Temp\ipykernel_3156\1770215685.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sales_data.loc[:, 'date'] = pd.to_datetime(sales_data['date'])
C:\Users\Admin\AppData\Local\Temp\ipykernel_3156\1770215685.py:2: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  sales_data.loc[:, 'date'] = pd.to_datetime(sales_data['date'])


In [19]:
# Sort the data by date
sales_data.sort_values(by=['item_id', 'store_id', 'date'], inplace=True)



C:\Users\Admin\AppData\Local\Temp\ipykernel_3156\3671283413.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sales_data.sort_values(by=['item_id', 'store_id', 'date'], inplace=True)


In [20]:
# Reset index
sales_data.reset_index(drop=True, inplace=True)

In [21]:
print(sales_data.head())


       item_id  item_id_sales store_id store_id_sales       date  sell_price  \
0  FOODS_1_001  HOBBIES_1_008     CA_1           CA_1 2012-12-29         0.5   
1  FOODS_1_001  HOBBIES_1_008     CA_1           CA_1 2012-12-30         0.5   
2  FOODS_1_001  HOBBIES_1_008     CA_1           CA_1 2012-12-31         0.5   
3  FOODS_1_001  HOBBIES_1_008     CA_1           CA_1 2013-01-01         0.5   
4  FOODS_1_001  HOBBIES_1_008     CA_1           CA_1 2013-01-02         0.5   

   d_1  d_2  d_3  d_4  ...  d_240  d_241  d_242  d_243  d_244  d_245  d_246  \
0    3    0    0    1  ...      0      0      0      0      0      0      0   
1    3    0    0    1  ...      0      0      0      0      0      0      0   
2    3    0    0    1  ...      0      0      0      0      0      0      0   
3    3    0    0    1  ...      0      0      0      0      0      0      0   
4    3    0    0    1  ...      0      0      0      0      0      0      0   

   d_247  d_248  d_249  
0      0      0    

 # Feature Engineering

In [26]:
print(events.head(4))

         date     event_name event_type
0  2011-02-06      SuperBowl   Sporting
1  2011-02-14  ValentinesDay   Cultural
2  2011-02-21  PresidentsDay   National
3  2011-03-09      LentStart  Religious


In [28]:
# Convert 'date' column in both DataFrames to datetime format
sales_data['date'] = pd.to_datetime(sales_data['date'])
events['date'] = pd.to_datetime(events['date'])

C:\Users\Admin\AppData\Local\Temp\ipykernel_3156\3375624579.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sales_data['date'] = pd.to_datetime(sales_data['date'])


In [29]:

# Create lag features for sales
for lag in range(1, 8):
    sales_data[f'lag_{lag}_sales'] = sales_data.groupby(['item_id', 'store_id'])['d_1'].shift(lag)


C:\Users\Admin\AppData\Local\Temp\ipykernel_3156\2796835567.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sales_data[f'lag_{lag}_sales'] = sales_data.groupby(['item_id', 'store_id'])['d_1'].shift(lag)
C:\Users\Admin\AppData\Local\Temp\ipykernel_3156\2796835567.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sales_data[f'lag_{lag}_sales'] = sales_data.groupby(['item_id', 'store_id'])['d_1'].shift(lag)
C:\Users\Admin\AppData\Local\Temp\ipykernel_3156\2796835567.py:3: SettingWithCopyWarning: 
A value

In [30]:
# Extract day of the week
sales_data['day_of_week'] = sales_data['date'].dt.dayofweek

# Create a feature for whether it's a weekend (0 for weekdays, 1 for weekends)
sales_data['is_weekend'] = (sales_data['day_of_week'] >= 5).astype(int)

# Merge with calendar events data
sales_data = sales_data.merge(events, on='date', how='left')

# Create an event indicator (1 if an event, 0 otherwise)
sales_data['event_indicator'] = sales_data['event_name'].notnull().astype(int)

C:\Users\Admin\AppData\Local\Temp\ipykernel_3156\4076940478.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sales_data['day_of_week'] = sales_data['date'].dt.dayofweek
C:\Users\Admin\AppData\Local\Temp\ipykernel_3156\4076940478.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sales_data['is_weekend'] = (sales_data['day_of_week'] >= 5).astype(int)


In [35]:
print(sales_data.head(10))

       item_id  item_id_sales store_id store_id_sales       date  sell_price  \
0  FOODS_1_001  HOBBIES_1_008     CA_1           CA_1 2012-12-29        0.50   
1  FOODS_1_001  HOBBIES_1_008     CA_1           CA_1 2012-12-30        0.50   
2  FOODS_1_001  HOBBIES_1_008     CA_1           CA_1 2012-12-31        0.50   
3  FOODS_1_001  HOBBIES_1_008     CA_1           CA_1 2013-01-01        0.50   
4  FOODS_1_001  HOBBIES_1_008     CA_1           CA_1 2013-01-02        0.50   
5  FOODS_1_001  HOBBIES_1_008     CA_1           CA_1 2013-01-03        0.50   
6  FOODS_1_001  HOBBIES_1_008     CA_1           CA_1 2013-01-04        0.50   
7  FOODS_1_001  HOBBIES_1_020     CA_2           CA_1 2012-12-15       11.68   
8  FOODS_1_001  HOBBIES_1_020     CA_2           CA_1 2012-12-16       11.68   
9  FOODS_1_001  HOBBIES_1_020     CA_2           CA_1 2012-12-17       11.68   

   d_1  d_2  d_3  d_4  ...  lag_3_sales  lag_4_sales  lag_5_sales  \
0    3    0    0    1  ...          0.0          0

In [36]:
sales_data['lag_1_sales'].fillna(0, inplace=True)
sales_data['lag_2_sales'].fillna(0, inplace=True)
sales_data['lag_3_sales'].fillna(0, inplace=True)
sales_data['lag_4_sales'].fillna(0, inplace=True)
sales_data['lag_5_sales'].fillna(0, inplace=True)
sales_data['lag_6_sales'].fillna(0, inplace=True)
sales_data['lag_7_sales'].fillna(0, inplace=True)

# Model Selection and Training

In [37]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

In [40]:
# Select features and target variable
features = ['sell_price', 'lag_1_sales', 'lag_2_sales', 'lag_3_sales', 'day_of_week', 'is_weekend', 'event_indicator']
target = 'd_1'

In [41]:
# Split the data into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(sales_data[features], sales_data[target], test_size=0.2, random_state=42)

In [57]:
# Initialize and train the model 
L_model = LinearRegression()
L_model.fit(X_train, y_train)

LinearRegression()

In [58]:
# Make predictions on the validation set
y_pred = model.predict(X_valid)

# Calculate Mean Absolute Error (MAE)
mae = mean_absolute_error(y_valid, y_pred)
print(f' linear regression MAE: {mae}')

 linear regression MAE: 0.31488925497608133


### Comparison Model

In [44]:
from sklearn.ensemble import RandomForestRegressor

# Initialize and train the Random Forest Regressor model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Make predictions on the validation set using the Random Forest model
rf_y_pred = rf_model.predict(X_valid)

# Calculate Mean Absolute Error (MAE) for the Random Forest model
rf_mae = mean_absolute_error(y_valid, rf_y_pred)
print(f'Random Forest MAE: {rf_mae}')


Random Forest MAE: 0.2338010426243403


In [61]:
# Save the trained model to a file
import joblib

joblib.dump(rf_model, 'best_random_forest_model.pkl')


joblib.dump(L_model, 'best_Linear_regression_model.pkl')



['best_Linear_regression_model.pkl']

#  Deployment as an API

In [54]:
pip install Flask


In [62]:
from flask import Flask, request, jsonify
import joblib  # Used for model persistence

app = Flask(__name__)



In [63]:
# Load your trained model 
rf_modelmodel = joblib.load('best_random_forest_model.pkl')

@app.route('/predict', methods=['POST'])
def predict_sales():
    try:
        # Get input data from JSON request
        data = request.get_json()

        # Prepare input data (must match the features used for training)
        input_features = [data['sell_price'], data['lag_1_sales'], data['lag_2_sales'],
                          data['lag_3_sales'], data['day_of_week'], data['is_weekend'],
                          data['event_indicator']]

        # Make a prediction using the loaded model
        predicted_sales = model.predict([input_features])[0]

        # Create a response JSON
        response = {'predicted_sales': predicted_sales}

        return jsonify(response)

    except Exception as e:
        return jsonify({'error': str(e)})

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (windowsapi)


SystemExit: 1

c:\Users\Admin\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3513: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
